# Data Background

Dataset is data engineering job postings from Glassdoor in the USA in March 2023. It includes company and job details such as company size, industry, location, title, salary, etc. 

Data source: https://github.com/Hamagistral/DataEngineers-Glassdoor/tree/master/data/kaggle

# Load and Examine Data

In [1]:
# import libraries

import numpy as np
import pandas as pd

In [2]:
SalaryData = pd.read_csv("glassdoor-data-engineer-kaggle.csv")
print('# of rows = ',SalaryData.shape[0])
SalaryData.head()

# of rows =  900


,company,company_rating,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,company_founded,company_revenue
0,PCS Global Tech\n4.7,4.7,"Riverside, CA",Data Engineer | PAID BOOTCAMP,Responsibilities\n· Analyze and organize raw d...,"$70,000 /yr (est.)",501 to 1000 Employees,Company - Private,Information Technology,Information Technology Support Services,NaN,Unknown / Non-Applicable
1,Futuretech Consultants LLC,NaN,"Newton, MS",Snowflake Data Engineer,My name is Dileep and I am a recruiter at Futu...,$42.50 /hr (est.),NaN,NaN,NaN,NaN,NaN,NaN
2,Clairvoyant\n4.4,4.4,Remote,Data Engineer (MDM),Required Skills:\nMust have 5-8+ Years of expe...,$67.50 /hr (est.),51 to 200 Employees,Company - Private,Pharmaceutical & Biotechnology,Biotech & Pharmaceuticals,NaN,Unknown / Non-Applicable
3,Apple\n4.2,4.2,"Cupertino, CA",Data Engineer,"Summary\nPosted: Dec 22, 2021\nWeekly Hours: 4...",NaN,10000+ Employees,Company - Public,Information Technology,Computer Hardware Development,1976.0,$10+ billion (USD)
4,Skytech Consultancy Services\n5.0,5.0,"Baltimore, MD",Data Engineer,Description of Work:\nTechnical experience in ...,$65.00 /hr (est.),1 to 50 Employees,Company - Public,NaN,NaN,NaN,Unknown / Non-Applicable


In [3]:
types = SalaryData.dtypes
print(types)
print("Keys of SalaryData dataset:\n", SalaryData.keys())

company              object
company_rating      float64
location             object
job_title            object
job_description      object
salary_estimate      object
company_size         object
company_type         object
company_sector       object
company_industry     object
company_founded     float64
company_revenue      object
dtype: object
Keys of SalaryData dataset:
 Index(['company', 'company_rating', 'location', 'job_title', 'job_description',
       'salary_estimate', 'company_size', 'company_type', 'company_sector',
       'company_industry', 'company_founded', 'company_revenue'],
      dtype='object')


In [4]:
SalaryData.isnull().sum()

company               0
company_rating      185
location              0
job_title             0
job_description       0
salary_estimate      64
company_size        135
company_type        135
company_sector      391
company_industry    391
company_founded     495
company_revenue     135
dtype: int64

# Data Cleaning

## 1. Covert All Salary to Annual

In [5]:
SalaryData['salary_estimate'].head()

0    $70,000 /yr (est.)
1     $42.50 /hr (est.)
2     $67.50 /hr (est.)
3                   NaN
4     $65.00 /hr (est.)
Name: salary_estimate, dtype: object

In [6]:
import re
def clean_salary(salary_string):
    if pd.isnull(salary_string):
        return np.nan
    else:
        match_year = re.search(r'\$(\d{1,3},?\d{0,3},?\d{0,3}) \/yr \(est.\)', salary_string)
        match_hour = re.search(r'\$(\d+(\.\d+)?) \/hr \(est.\)', salary_string)

        if match_year:
            salary_amount = float(match_year.group(1).replace(',',''))
        elif match_hour:
            hourly_salary = float(match_hour.group(1))
            salary_amount = hourly_salary *1800
        else:
            salary_amount = np.nan
        return salary_amount

In [7]:
SalaryData['salary_estimate'] = SalaryData['salary_estimate'].apply(clean_salary)
SalaryData['salary_estimate'].head()

0     70000.0
1     76500.0
2    121500.0
3         NaN
4    117000.0
Name: salary_estimate, dtype: float64

## 2. Clean Location And Extract States

In [8]:
SalaryData['location'] = SalaryData['location'].astype(str)
SalaryData['job_state'] = np.where(SalaryData['location'].str.lower() == 'remote', SalaryData['location'], SalaryData['location'].str.split(', ').str[-1])
SalaryData.job_state.value_counts()

Remote           149
GA                96
TX                94
CA                87
NJ                80
MN                49
DC                46
VA                44
WI                36
MD                34
IL                34
MS                24
NY                21
MA                19
CT                18
OR                17
United States     14
PA                12
UT                 8
TN                 6
FL                 4
OH                 3
DE                 1
SC                 1
OK                 1
CO                 1
NC                 1
Name: job_state, dtype: int64

## 3. Clean Job Title

### How do titles look like?

In [9]:
sorted(SalaryData['job_title'].unique())

['AWS Data Engineer',
 'AWS Senior Data Engineer',
 'AWS python data engineer',
 'Anaplan Data Engineer',
 'Azure Data Engineer',
 'Azure Data Engineer (Azure Data Factory, Databricks)',
 'Azure Tech Lead/ Sr Data Engineer',
 'Big Data Engineer',
 'Big Data SDET Engineer',
 'Big Data Sytems Engineer(Hadoop)',
 'Cloud Data Engineer (Azure)',
 'Data Analytics Engineer',
 'Data Engineer',
 'Data Engineer (ETL & Data Catalogue Support)',
 'Data Engineer (ETL)',
 'Data Engineer (MDM)',
 'Data Engineer (Remote)',
 'Data Engineer - Hybrid Onsite - King of Prussia, PA',
 'Data Engineer - Onsite',
 'Data Engineer - Remote',
 'Data Engineer 925',
 'Data Engineer ETL',
 'Data Engineer | PAID BOOTCAMP',
 'Data Engineer- Remote',
 'Data Engineer/Data Scientist',
 'Data Integration Engineer',
 'Data Logging Engineer',
 'Data Software Engineer',
 'Data Warehouse Engineer',
 'Data engineer lead',
 'ETL Data Engineer',
 'Full Stack Data Engineer',
 'GCP Data Engineer',
 'Hadoop Big Data Engineer',
 'Jr

In [10]:
print('Contains "data engineer" = ', SalaryData['job_title'].str.contains('data engineer', case=False).sum())
print('Not contains "data engineer" = ', (~SalaryData['job_title'].str.contains('data engineer', case=False)).sum())

Contains "data engineer" =  845
Not contains "data engineer" =  55


### Remove non-data engineer job postings

In [11]:
SalaryData = SalaryData[SalaryData['job_title'].str.contains('data engineer', case=False)]
print('# of rows = ',SalaryData.shape[0])

# of rows =  845


### Categorize by seniority

In [12]:
def seniority (title):
    
    # considered senior if title has sr., senior, lead or principal
    if'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'junior' in title.lower():
        return 'junior'
    else:
        return 'intermediate'

In [13]:
SalaryData['seniority'] = SalaryData['job_title'].apply(seniority)
SalaryData.seniority.value_counts()

intermediate    626
senior          210
junior            9
Name: seniority, dtype: int64

## 4. Extract Skills From Job Description

### How do job descriptions look like?

In [14]:
sorted(SalaryData['job_description'].unique())

['(Submit Resume and All Inclusive hourly rate or Salary).\nThe selected candidate will coordinate the maintenance and ongoing development of data systems for two closely related projects as the primary data engineer on a small (2-5) technical services team. The existing system has been developed in Microsoft SQL hosted via Microsoft Azure Government Cloud. The candidate does NOT need to be Department of Defense system qualified. Work is primarily remote but the candidate should reside in Illinois or the region; work teams meet once per month for strategic planning in DeKalb, Illinois.\no Preferred skill set:\n§ Strong background in Microsoft SQL Server, both installing/configuring and administering.\n§ Experience with Microsoft Azure.\n§ Experience designing and implementing ETL processes.\n§ Experience with event-driven automated systems preferred.\n§ Some familiarity with Tableau Server and Office 365 preferred.\n§ Strong communication skills\nJob Types: Full-time, Contract\nSchedul

### List top skills by type

In [15]:
prog_languages = ['python', 'java', 'scala', 'go', 'r', 'c++', 'c#', 'sql', 'nosql', 'rust', 'shell']
cloud_tools = ['aws', 'azure', 'google cloud', 'snowflake', 'databricks', 'redshift']
viz_tools = ['power bi', 'tableau', 'excel', 'ssis', 'qlik', 'sap', 'looker']
databases = ['sql server', 'postgresql', 'mongodb', 'mysql', 'oracle', 'casandra', 'elasticsearch', 'dynamodb', 'snowflake', 'redis', 'neo4j', 'hive', 'dbt']
big_data = ['spark', 'hadoop', 'kafka', 'flink']
devops = ['gitlab', 'terraform', 'docker', 'bash', 'ansible']

### Match keywords in each skill list and create columns for each skill type

In [16]:
def extract_keywords(description, keywords):
    pattern = r'\b(?:{})\b'.format('|'.join(map(re.escape, keywords)))
    matches = set(re.findall(pattern, description.lower(), flags=re.IGNORECASE))
    
    return list(matches)

In [17]:
SalaryData['job_languages'] = SalaryData['job_description'].apply(lambda x: extract_keywords(x, prog_languages))
SalaryData['job_cloud'] = SalaryData['job_description'].apply(lambda x: extract_keywords(x, cloud_tools))
SalaryData['job_viz'] = SalaryData['job_description'].apply(lambda x: extract_keywords(x, viz_tools))
SalaryData['job_databases'] = SalaryData['job_description'].apply(lambda x: extract_keywords(x, databases))
SalaryData['job_bigdata'] = SalaryData['job_description'].apply(lambda x: extract_keywords(x, big_data))
SalaryData['job_devops'] = SalaryData['job_description'].apply(lambda x: extract_keywords(x, devops))

## 5. Extract Education Level From Job Description

### List education levels

In [18]:
education = ['associate', 'bachelor', 'master', 'phd']

In [19]:
# if matches multiple degree keywords, count only the lowest degree

def extract_degree(description, degrees):
    pattern = r'\b(?:{})\b'.format('|'.join(map(re.escape, degrees)))
    matches = re.findall(pattern, description.lower(), flags=re.IGNORECASE)
    
    if 'bachelor' in matches:
        return 'bachelor'
    elif 'master' in matches:
        return 'master'
    elif 'phd' in matches:
        return 'phd'
    
    return None   

In [20]:
SalaryData['job_education'] = SalaryData['job_description'].apply(lambda x:extract_degree(x, education))
SalaryData['job_education'].value_counts()

bachelor    253
master       34
Name: job_education, dtype: int64

### Check if there are other ways specifying education level

In [21]:
# Function to split text into sentences and find the sentence containing "degree"
def extract_degree_sentence(text):
    sentences = re.split(r'[\n.]+', text)  # Split text by "\n" and "."
    degree_sentence = next((sentence for sentence in sentences if re.search(r'\bdegree\b', sentence, re.IGNORECASE)), None)
    
    return degree_sentence

# Apply the function to the 'job_description' column
SalaryData['degree_sentence'] = SalaryData['job_description'].apply(extract_degree_sentence)

# Print the unique sentence containing "degree"
degree_sentence = SalaryData['degree_sentence'].dropna().unique()
for sentence in degree_sentence:
    print(sentence)

· Degree in Computer Science, IT, or similar field (STEM)
Bachelor’s Degree in Computer Science, IT, or related field
Bachelor's or master's degree in a training related field or 8 years of experience in lieu of a degree
Bachelor's degree in Computer Science, Mathematics, Statistics or related experience
Applicant must have a Bachelor’s degree in Computer Science, Information Systems, or Information Technology and 5 years of progressive post-baccalaureate experience in the job offered or a related occupation
EDUCATION: Bachelors Degree in Mathematics/Statistics/Technology/Science/Engineering/Applied Mathematics or related field
BS/MS degree in Computer Science or equivalent proven experience
Bachelor's or Master's degree in Computer Science, Information Systems, Engineering or equivalent
(1) A bachelor's degree, or (2) achievement of formal certifications recognized in the industry as equivalent to a bachelor's degree (e
Bachelors Degree
Bachelor's degree in computer science, Math, Ana

### To capture more education requirement, add BS/MS and "degree in"

In [22]:
education = ['associate', 'bs', 'bachelor', 'ms', 'master', 'phd', 'degree in']

In [23]:
# if matches multiple degree keywords, count only the lowest degree

def extract_degree_2(description, degrees):
    pattern = r'\b(?:{})\b'.format('|'.join(map(re.escape, degrees)))
    matches = re.findall(pattern, description.lower(), flags=re.IGNORECASE)
    
    # if degree is mentioned but level is not specified, default bachelor
    if 'bs' in matches or 'bachelor' in matches or 'degree in' in matches:
        return 'bachelor'
    elif 'ms' in matches or 'master' in matches:
        return 'master'
    elif 'phd' in matches:
        return 'phd'
    
    return None   

In [24]:
SalaryData['job_education'] = SalaryData['job_description'].apply(lambda x:extract_degree_2(x, education))
SalaryData['job_education'].value_counts()

bachelor    287
master       35
Name: job_education, dtype: int64

## 6. Categorize Experience Level

### Check descriptions with keyword "year"

In [25]:
# Function to split text into sentences and find the sentence containing "year"
def extract_exp_sentence(text):
    sentences = re.split(r'[\n.]+', text)  # Split text by "\n" and "."
    exp_sentence = next((sentence for sentence in sentences if re.search(r'\byear\b', sentence, re.IGNORECASE)), None)
    
    return exp_sentence

# Apply the function to the 'job_description' column
SalaryData['exp_sentence'] = SalaryData['job_description'].apply(extract_exp_sentence)

# Print the unique sentence containing "degree"
exp_sentence = SalaryData['exp_sentence'].dropna().unique()
for sentence in exp_sentence:
    print(sentence)

00 per year
MDM (Master Data Management): 1 year (Required)
 The base pay for this position ranges from $105,700/year in our lowest geographic market up to $205,600/year in our highest geographic market
NoSQL: 1 year (Preferred)
3+ year’s working with multiple Big Data file formats (Parquet, Avro, Delta Lake)
 Each year we also meet in person for an all-expenses-paid annual retreat as a team
Paid-time off (PTO), accruing with each year of service, up to 20 days, plus 11 paid holidays
Esri’s competitive total rewards strategy includes industry-leading health and welfare benefits: medical, dental, vision, basic and supplemental life insurance for employees (and their families), 401(k) and profit-sharing programs, minimum accrual of 80 hours of vacation leave, twelve paid holidays throughout the calendar year, and opportunities for personal and professional growth
1+ year experience working with Azure analytical stack
12+ years of experience needed to serve as the subject matter expert on

### Extract number from year of experience

In [26]:
def extract_experience(description):
    # match the number before "years of experience", "year of experience", "year's working" etc.
    # excluding "yearly" because it would be salary instead of experience
    pattern = r'\b(\d+)\b\s*(?:year\'s\s+working|years?\s+(?:of\s+)?experience)\b(?!ly)'
    matches = re.findall(pattern, description, flags=re.IGNORECASE)
    
    sentences = re.split(r'[\n.]+', description)  # Split description into sentences
    matched_sentences = [sentence.strip() for sentence in sentences if re.search(pattern, sentence, flags=re.IGNORECASE)]

    return matches, matched_sentences

SalaryData['job_description'].apply(lambda x: extract_experience(x)[0]).explode().unique()

array([nan, '2', '8', '3', '10', '5', '6', '80', '7', '4', '1', '15'],
      dtype=object)

In [27]:
def extract_experience(description):
    pattern = r'(\d+)\+?(?:\s*year(?:\'s)?\s*(?:of)?\s*(?:working\s*)?(?:experience)?)'
    matches = re.findall(pattern, description, flags=re.IGNORECASE)
    
    if matches:
        experience = matches[0]
        if int(experience) < 2:
            return "0-2 years"
        elif int(experience) < 5:
            return "2-5 years"
        elif int(experience) < 10:
            return "5-10 years"
        else:
            return "10+ years"
    else:
        return None

In [28]:
SalaryData['job_experience'] = SalaryData['job_description'].apply(lambda x: extract_experience(x))
SalaryData['job_experience'].value_counts()

5-10 years    315
2-5 years     118
10+ years      93
0-2 years      85
Name: job_experience, dtype: int64

# Export Data

In [29]:
SalaryData.head()

,company,company_rating,location,job_title,job_description,salary_estimate,company_size,company_type,company_sector,company_industry,...,job_languages,job_cloud,job_viz,job_databases,job_bigdata,job_devops,job_education,degree_sentence,exp_sentence,job_experience
0,PCS Global Tech\n4.7,4.7,"Riverside, CA",Data Engineer | PAID BOOTCAMP,Responsibilities\n· Analyze and organize raw d...,70000.0,501 to 1000 Employees,Company - Private,Information Technology,Information Technology Support Services,...,"[sql, python, java]",[],[],[],[],[],bachelor,"· Degree in Computer Science, IT, or similar f...",00 per year,0-2 years
1,Futuretech Consultants LLC,NaN,"Newton, MS",Snowflake Data Engineer,My name is Dileep and I am a recruiter at Futu...,76500.0,NaN,NaN,NaN,NaN,...,[sql],[snowflake],[ssis],[snowflake],[],[],bachelor,"Bachelor’s Degree in Computer Science, IT, or ...",None,5-10 years
2,Clairvoyant\n4.4,4.4,Remote,Data Engineer (MDM),Required Skills:\nMust have 5-8+ Years of expe...,121500.0,51 to 200 Employees,Company - Private,Pharmaceutical & Biotechnology,Biotech & Pharmaceuticals,...,"[sql, python]","[databricks, aws]",[],[],[spark],[],master,None,MDM (Master Data Management): 1 year (Required),5-10 years
3,Apple\n4.2,4.2,"Cupertino, CA",Data Engineer,"Summary\nPosted: Dec 22, 2021\nWeekly Hours: 4...",NaN,10000+ Employees,Company - Public,Information Technology,Computer Hardware Development,...,[python],[],[tableau],[],[],[],None,None,None,None
4,Skytech Consultancy Services\n5.0,5.0,"Baltimore, MD",Data Engineer,Description of Work:\nTechnical experience in ...,117000.0,1 to 50 Employees,Company - Public,NaN,NaN,...,[sql],[],[tableau],[oracle],[],[],bachelor,Bachelor's or master's degree in a training re...,None,5-10 years


In [30]:
SalaryData.to_csv("glassdoor-data-engineer-kaggle_cleaned.csv", index = False)